<a href="https://colab.research.google.com/github/arthurcamargotradingcomdados/tutoriais_publicos/blob/main/Rebalanceamento_de_Carteiras_16_03_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install vectorbt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 2.0 MB/s eta 0:00:00
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.9.3
    Uninstalling beautifulsoup4-4.9.3:
      Successfully uninstalled beautifulsoup4-

##Buy and Hold

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

In [3]:
ativos = ['PETR4.SA', 'WEGE3.SA', 'ITSA4.SA', 'BBDC4.SA', 'BRFS3.SA']
start = '2018-01-01'
end = '2020-01-01'

vbt.settings.array_wrapper['freq'] = 'days'
vbt.settings.returns['year_freq'] = '252 days'
vbt.settings.portfolio.stats['incl_unrealized'] = True

In [4]:
#Obter os dados

precos = yf.download(ativos, start=start, end=end)['Close']

[*********************100%***********************]  5 of 5 completed


In [5]:
#Obter os retornos

retornos = precos.pct_change()

In [6]:
precos

,BBDC4.SA,BRFS3.SA,ITSA4.SA,PETR4.SA,WEGE3.SA
Date,,,,,
2018-01-02,19.693554,37.270000,8.681621,16.549999,9.623076
2018-01-03,19.773239,37.000000,8.783943,16.700001,9.476923
2018-01-04,20.097670,37.400002,8.949232,16.730000,9.384615
2018-01-05,20.211506,39.240002,8.964974,16.830000,9.423076
2018-01-08,20.205814,39.700001,8.957103,17.030001,9.500000
...,...,...,...,...,...
2019-12-20,26.814425,34.810001,11.991341,30.260000,16.945000
2019-12-23,26.979713,34.529999,12.077922,30.500000,16.895000
2019-12-26,27.317806,35.180000,12.285714,30.910000,17.424999


In [7]:
retornos

,BBDC4.SA,BRFS3.SA,ITSA4.SA,PETR4.SA,WEGE3.SA
Date,,,,,
2018-01-02,NaN,NaN,NaN,NaN,NaN
2018-01-03,0.004046,-0.007244,0.011786,0.009064,-0.015188
2018-01-04,0.016408,0.010811,0.018817,0.001796,-0.009740
2018-01-05,0.005664,0.049198,0.001759,0.005977,0.004098
2018-01-08,-0.000282,0.011723,-0.000878,0.011884,0.008163
...,...,...,...,...,...
2019-12-20,-0.020850,0.012802,-0.009299,-0.011434,0.008631
2019-12-23,0.006164,-0.008044,0.007220,0.007931,-0.002951
2019-12-26,0.012531,0.018824,0.017204,0.013443,0.031370


In [8]:
#Pesos

pesos = np.array([0.2, 0.2, 0.2,0.2,0.2])

In [9]:
#Definir tamanho das ordens

bh_size = np.full_like(precos, np.nan)

In [11]:
print(bh_size.shape)

(494, 5)


In [12]:
#Na primeira linha desse array coloque os pesos
bh_size[0,:] = pesos

In [14]:
bh_size

array([[0.2, 0.2, 0.2, 0.2, 0.2],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       ...,
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]])

In [45]:
bh_size.shape

(494, 5)

In [15]:
#Rodar o backtest

bh_pf = vbt.Portfolio.from_orders(
    close=precos,
    size=bh_size,
    size_type='targetpercent',
    group_by=True,
    cash_sharing=True
)

In [16]:
print(len(bh_pf.orders))

5


In [17]:
print(bh_pf.stats())

Start                         2018-01-02 00:00:00
End                           2019-12-30 00:00:00
Period                          494 days 00:00:00
Start Value                                 100.0
End Value                              147.079352
Total Return [%]                        47.079352
Benchmark Return [%]                    47.079352
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        31.215395
Max Drawdown Duration           230 days 00:00:00
Total Trades                                    5
Total Closed Trades                             0
Total Open Trades                               5
Open Trade PnL                          47.079352
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


In [ ]:
def plot_allocation(rb_pf):
    rb_asset_value = rb_pf.asset_value(group_by=False)
    rb_value = rb_pf.value()
    rb_idxs = np.flatnonzero((rb_pf.asset_flow() != 0).any(axis=1))
    rb_dates = rb_pf.wrapper.index[rb_idxs]
    fig = (rb_asset_value.vbt / rb_value).vbt.plot(
        trace_names=ativos,
        trace_kwargs=dict(
            stackgroup='one'
        )
    )
    for rb_date in rb_dates:
        fig.add_shape(
            dict(
                xref='x',
                yref='paper',
                x0=rb_date,
                x1=rb_date,
                y0=0,
                y1=1,
                line_color=fig.layout.template.layout.plot_bgcolor
            )
        )
    fig.show()

In [30]:
plot_allocation(bh_pf)

##Rebalanceamento

In [20]:
precos.index.to_period('m')

PeriodIndex(['2018-01', '2018-01', '2018-01', '2018-01', '2018-01', '2018-01',
             '2018-01', '2018-01', '2018-01', '2018-01',
             ...
             '2019-12', '2019-12', '2019-12', '2019-12', '2019-12', '2019-12',
             '2019-12', '2019-12', '2019-12', '2019-12'],
            dtype='period[M]', name='Date', length=494)

In [22]:
mascara_rb = ~precos.index.to_period('m').duplicated()

In [23]:
mascara_rb

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False,

In [24]:
mascara_rb.shape

(494,)

In [25]:
tamanho_rb = np.full_like(precos, np.nan)
tamanho_rb

array([[nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       ...,
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]])

In [27]:
tamanho_rb[mascara_rb,:] = pesos
tamanho_rb

array([[0.2, 0.2, 0.2, 0.2, 0.2],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       ...,
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]])

In [28]:
tamanho = pd.DataFrame(tamanho_rb)
tamanho

,0,1,2,3,4
0,0.2,0.2,0.2,0.2,0.2
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
489,NaN,NaN,NaN,NaN,NaN
490,NaN,NaN,NaN,NaN,NaN
491,NaN,NaN,NaN,NaN,NaN
492,NaN,NaN,NaN,NaN,NaN


In [31]:
#Construir o portfolio

rebalance_pf = vbt.Portfolio.from_orders(
    close=precos,
    size=tamanho_rb,
    size_type='targetpercent',
    group_by=True,
    cash_sharing=True,
    call_seq='auto'

)

In [32]:
print(rebalance_pf.stats())

Start                                 2018-01-02 00:00:00
End                                   2019-12-30 00:00:00
Period                                  494 days 00:00:00
Start Value                                         100.0
End Value                                      152.355431
Total Return [%]                                52.355431
Benchmark Return [%]                            47.079352
Max Gross Exposure [%]                              100.0
Total Fees Paid                                       0.0
Max Drawdown [%]                                30.677331
Max Drawdown Duration                   231 days 00:00:00
Total Trades                                           62
Total Closed Trades                                    57
Total Open Trades                                       5
Open Trade PnL                                  42.468584
Win Rate [%]                                    77.192982
Best Trade [%]                                  65.661294
Worst Trade [%

In [33]:
rebalance_pf.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,BBDC4.SA,2018-01-02,1.015561,19.693554,0.0,Buy
1,1,BRFS3.SA,2018-01-02,0.536625,37.270000,0.0,Buy
2,2,ITSA4.SA,2018-01-02,2.303717,8.681621,0.0,Buy
3,3,PETR4.SA,2018-01-02,1.208459,16.549999,0.0,Buy
4,4,WEGE3.SA,2018-01-02,2.078337,9.623076,0.0,Buy
...,...,...,...,...,...,...,...
115,115,WEGE3.SA,2019-12-02,0.276925,15.080000,0.0,Sell
116,116,BRFS3.SA,2019-12-02,0.013021,36.419998,0.0,Sell
117,117,ITSA4.SA,2019-12-02,0.115025,11.567099,0.0,Buy
118,118,BBDC4.SA,2019-12-02,0.061292,25.236664,0.0,Buy


In [40]:
rebalance_pf.value()

Date
2018-01-02    100.000000
2018-01-03    100.049273
2018-01-04    100.818587
2018-01-05    102.158630
2018-01-08    102.783130
                 ...    
2019-12-20    150.460375
2019-12-23    150.785731
2019-12-26    153.623887
2019-12-27    153.503375
2019-12-30    152.355431
Name: group, Length: 494, dtype: float64

In [42]:
rebalance_pf.asset_value(group_by=False)

,BBDC4.SA,BRFS3.SA,ITSA4.SA,PETR4.SA,WEGE3.SA
Date,,,,,
2018-01-02,20.000000,20.000000,20.000000,20.000000,20.000000
2018-01-03,20.080925,19.855111,20.235722,20.181271,19.696244
2018-01-04,20.410404,20.069762,20.616501,20.217523,19.504397
2018-01-05,20.526012,21.057151,20.652767,20.338369,19.584331
2018-01-08,20.520232,21.303998,20.634633,20.580062,19.744206
...,...,...,...,...,...
2019-12-20,30.628209,27.551750,29.883287,30.006059,32.391070
2019-12-23,30.817007,27.330131,30.099054,30.244045,32.295494
2019-12-26,31.203186,27.844600,30.616887,30.650604,33.308609


In [43]:
plot_allocation(rebalance_pf)